# Installing importing libraries


In [ ]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 10.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 62.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.9/129.9 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 9.5 MB/s 

# Importing Libraries

In [ ]:
import gradio as gr
import tensorflow as tf
import numpy as np
import pandas as pd
import requests
import keras

from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing import image
from PIL import Image
import cv2

# Importing the Model and get the songs csv

In [ ]:
model = keras.saving.load_model('./drive/MyDrive/Final_ResNet50_Model.keras')
songs = pd.read_csv('./drive/MyDrive/songs_mood.csv')

# Code to Utilise web cam for input of images.
Since it was throwing errors half the time and working the other half, this feature was put on hold

In [ ]:
"""import imutils
detection_model_path = './drive/MyDrive/haarcascade_frontalface_default.xml'
face_detection = cv2.CascadeClassifier(detection_model_path)
emotion_labels = ['Angry', 'Fear', 'Happy', 'Neutral', 'Sad']
def predict_cam(frame):

    frame = imutils.resize(frame, width=300)
    gray = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
    faces = face_detection.detectMultiScale(gray, scaleFactor=1.1,
                                            minNeighbors=5, minSize=(30, 30),
                                            flags=cv2.CASCADE_SCALE_IMAGE)

    frameClone = frame.copy()
    if len(faces) > 0:
        faces = sorted(faces, reverse=True,
                       key=lambda x: (x[2] - x[0]) * (x[3] - x[1]))[0]
        (fX, fY, fW, fH) = faces
        # Extract the ROI of the face from the grayscale image, resize it to a fixed 28x28 pixels, and then prepare
        # the ROI for classification via the CNN
        roi = gray[fY:fY + fH, fX:fX + fW]
        roi = cv2.resize(roi, (64, 64))
        roi = roi.astype("float") / 255.0
        roi = img_to_array(roi)
        roi = np.expand_dims(roi, axis=0)

        preds = model.predict(roi)[0]
        label = emotion_labels[preds.argmax()]
    else:
        return frameClone, "Can't find your face"

    probs = {}
    cv2.putText(frameClone, label, (fX, fY - 10),
                cv2.FONT_HERSHEY_DUPLEX, 1, (238, 164, 64), 1)
    cv2.rectangle(frameClone, (fX, fY), (fX + fW, fY + fH),
                  (238, 164, 64), 2)

    for (i, (emotion, prob)) in enumerate(zip(emotion_labels, preds)):
        probs[emotion] = float(prob)

    return frameClone, probs

cam_interface = gr.Interface(
    fn = predict_cam,
    inputs = gr.Image(label = 'Image',sources = ['webcam']),
    outputs = [gr.Image(label="Predicted Emotion"),
    gr.Label(num_top_classes=3, label="Top 3 Probabilities")],
    title = 'VibeSync',
    description = 'Facial Emotion Based Music Recommendation System'
)
interface = gr.TabbedInterface(
    [upload_interface,cam_interface],
    ["Upload Image","Web Cam Feed"]
)"""

In [ ]:
emotion_labels = {'angry': 0, 'fear': 1, 'happy': 2, 'neutral': 3, 'sad': 4}
index_to_emotion = {v: k for k, v in emotion_labels.items()}
emotion_reference = {'happy':'Happy','sad':'Sad','neutral':'Calm','angry':'Energetic','fear':'Energetic'}

{0: 'angry', 1: 'fear', 2: 'happy', 3: 'neutral', 4: 'sad'}

# Deploying the model and the Backend of the software.

In [ ]:
def prepare_image(img_pil):
  img = img_pil.resize((224,224))
  img_array = img_to_array(img)
  img_array = np.expand_dims(img_array, axis=0)  # Convert single image to a batch.
  img_array /= 255.0
  return img_array

def predict_emotion(image):
  processed_image = prepare_image(image)
  prediction = model.predict(processed_image)
  predicted_class = np.argmax(prediction, axis=1)
  predicted_emotion = index_to_emotion.get(predicted_class[0],'Unknown Class')
  return predicted_emotion

def recommend_music(image):
  # predict emotion
  emotion = predict_emotion(image)
  emotion_ref = emotion_reference[emotion]

  # fetch songs list corresponding to emotions
  songs_list = songs[songs['mood'] == emotion_ref]
  songs_list = songs_list.sample(n=5)
  songs_list = songs_list.reset_index(drop=True)

  # converting songs list to hyperlinks to send it to output
  songs_html = "<ul>"
  for x, row in songs_list.iterrows():
    song_url = f'spotify:track:{row["id"]}'
    song_name = f"{row['name']} by {row['artist']}"
    songs_html += f"<li><a href='{song_url}'>{song_name}</a></li>"
  songs_html += "</ul>"
  return emotion, songs_html

# Interface of the software made using gradio

In [ ]:
upload_interface = gr.Interface(
    fn = recommend_music,
    inputs = gr.Image(type='pil',label = 'Image',sources=["upload","clipboard"]),
    outputs=[gr.Textbox(label='Emotion'), gr.HTML(label='Recommended Songs')],
    title = 'VibeSync',
    description = 'Facial Emotion Based Music Recommendation System'
)
upload_interface.launch(debug = True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://5926d33574a3787f72.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


1/1 [==============================] - 0s 276ms/step
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7861 <> https://5926d33574a3787f72.gradio.live
